## Clean Version

### Louvain Baseline (idx: 0)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 
import time

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
data_names = ['Zheng']
#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)
    
    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    #data_for_excel['fr_tp'] = FR_tp
    data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 0
    data_for_excel['method_name'] = 'Louv_baseline'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:
                ''' 
                Louvain on G (baseline)
                '''
                label_, NMI_, Purity_ = do_Louvain(G, G, res, 0, label)
                Num_Cluster_Louv_on_G = len(set(label_))
                
                data_for_excel[res]['FL'] = [round(NMI_,2), round(Purity_,2), '100%', Num_Cluster_Louv_on_G]
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



### Leiden baseline (idx: 1)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    #data_for_excel['fr_tp'] = FR_tp
    data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 1
    data_for_excel['method_name'] = 'Leiden_baseline'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:
                '''
                Leiden on G (Baseline)
                '''
                label_, NMI_, Purity_ = do_Leiden(G, G, res, 0, label)
                Num_Cluster_Leiden_on_G = len(set(label_))
                
                data_for_excel[res]['FL'] = [round(NMI_,2), round(Purity_,2), '100%', Num_Cluster_Leiden_on_G]
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



### Leiden + Majority vote (One round) (2)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    data_for_excel['fr_tp'] = FR_tp
    #data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 2
    data_for_excel['method_name'] = 'Leiden + Vote(1_Round)'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, _, _ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Leid_on_H = len({x for x in set(H_label) if x != -1})
                    
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes, one_round=True)
                    selected_node_num = len([x for x in H_label if x != -1])
                    selected_node_percent = str(round(selected_node_num*100/(G.number_of_nodes()),2))+'%' 
                    data_for_excel[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), selected_node_percent, Num_Cluster_Leid_on_H]
                    
                    
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



### Leiden + Majority vote (3)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    data_for_excel['fr_tp'] = FR_tp
    #data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 3
    data_for_excel['method_name'] = 'Leiden + Vote'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, _, _ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Leid_on_H = len({x for x in set(H_label) if x != -1})
                    
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num = len([x for x in H_label if x != -1])
                    selected_node_percent = str(round(selected_node_num*100/(G.number_of_nodes()),2))+'%' 
                    data_for_excel[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), selected_node_percent, Num_Cluster_Leid_on_H]
                    
                    
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



### Leiden + Majority vote + Leiden (4)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    data_for_excel['fr_tp'] = FR_tp
    #data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 4
    data_for_excel['method_name'] = 'Leiden + Vote + Leiden'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, _, _ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Leid_on_H = len({x for x in set(H_label) if x != -1})
                    
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num = len([x for x in H_label if x != -1])
                    selected_node_percent = str(round(selected_node_num*100/(G.number_of_nodes()),2))+'%' 
                    
                    '''
                    Leiden again on the vote subgraph
                    '''
                    selected_node_list = [node_num_ for node_num_, node_label_ in enumerate(H_label) if node_label_ != -1]
                    H2 = getInducedSubgraph(G, selected_node_list)
                    H2_ig, ig_to_nx, nx_to_ig = networkx_to_igraph(H2) # Mapping = igraph index to networkx index
                    initial_membership = [H_label[ig_to_nx[i]] for i in list(H2_ig.vs.indices)]
                    partition2_ = la.find_partition(H2_ig, la.RBConfigurationVertexPartition, initial_membership = initial_membership, n_iterations=-1, resolution_parameter=res, seed=0)
                    H_label2 = part_to_full_label(partition2_, G.number_of_nodes(), ig_to_nx)
                    
                    label_compressed2, H_label_compressed2 = get_compressed_labels(label,H_label2, selected_node_list)
                    NMI_louv_vote_louv = NMI(H_label_compressed2, label_compressed2)
                    Purity_louv_vote_louv = met.purity_score(label_compressed2, H_label_compressed2)
                    Num_Cluster_Leid_on_H2 = len({x for x in set(H_label2) if x != -1})
                    
                    
                    data_for_excel[res][FR_tp[FR_type]] = [round(NMI_louv_vote_louv,2), round(Purity_louv_vote_louv,2), selected_node_percent, Num_Cluster_Leid_on_H2]
                    
                    
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



### Leiden + Vote + Leiden + Vote (5)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = G.copy()
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    data_for_excel['fr_tp'] = FR_tp
    #data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 5
    data_for_excel['method_name'] = '(Leiden + Vote)x2'
    data_for_excel['num_comm'] = True_num_of_clusters

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, _, _ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Leid_on_H = len({x for x in set(H_label) if x != -1})
                    
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num = len([x for x in H_label if x != -1])
                    
                    
                    '''
                    Leiden again on the vote subgraph
                    '''
                    selected_node_list = [node_num_ for node_num_, node_label_ in enumerate(H_label) if node_label_ != -1]
                    H2 = getInducedSubgraph(G, selected_node_list)
                    H2_ig, ig_to_nx, nx_to_ig = networkx_to_igraph(H2) # Mapping = igraph index to networkx index
                    initial_membership = [H_label[ig_to_nx[i]] for i in list(H2_ig.vs.indices)]
                    partition2_ = la.find_partition(H2_ig, la.RBConfigurationVertexPartition, initial_membership = initial_membership, n_iterations=-1, resolution_parameter=res, seed=0)
                    H_label2 = part_to_full_label(partition2_, G.number_of_nodes(), ig_to_nx)
                    
                    label_compressed2, H_label_compressed2 = get_compressed_labels(label,H_label2, selected_node_list)
                    NMI_louv_vote_louv = NMI(H_label_compressed2, label_compressed2)
                    Purity_louv_vote_louv = met.purity_score(label_compressed2, H_label_compressed2)
                    Num_Cluster_Leid_on_H2 = len({x for x in set(H_label2) if x != -1})
                    
                    '''
                    Leiden + vote + Leiden + vote
                    '''
                    nodes_rest2 = [node_num_ for node_num_, node_label_ in enumerate(H_label2) if node_label_ == -1]
                    NMI_List2, Purity_List2, Balance_List2, Preserv_List2, InEdge_List2 = merge_by_vote(selected_node_list, nodes_rest2, H_label2, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num2 = len([node_num_ for node_num_, node_label_ in enumerate(H_label2) if node_label_ != -1])
                    selected_node_percent = str(round(selected_node_num2*100/(G.number_of_nodes()),2))+'%' 

                    data_for_excel[res][FR_tp[FR_type]] = [round(NMI_List2[-1],2), round(Purity_List2[-1],2), selected_node_percent, Num_Cluster_Leid_on_H2]
                    
                    
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
    
        
            

                                
                    

                    
                    



### Leiden + Strong Majority Vote + Random Walk (Weak Majority Vote) (idx: 6)

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 
import copy

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05,0.25, 0.5, 1, 1.5, 5]

for data_idx, data in enumerate(data_names): 
    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/(Vote+RandWalk)'+data+'.pdf'
    data_for_plot = defaultdict(lambda: defaultdict(dict)) #nested dictionary to store data for plots

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    dictionary for exporting to excel
    '''
    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    data_for_excel['fr_tp'] = FR_tp
    #data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 6
    data_for_excel['method_name'] = 'Leiden_Vote_RW(weak_vote)'
    data_for_excel['num_comm'] = True_num_of_clusters

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = copy.deepcopy(G)
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)
    print('Number of nodes:',G.number_of_nodes())

    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            for Mod_type in [0]:
                ''' 
                Louvain on G (baseline)
                '''
                label_, NMI_, Purity_ = do_Louvain(G, G, res, 0, label)
                Num_Cluster_Louv_on_G = len(set(label_))

                '''
                Leiden on G (Baseline)
                '''
                label_, NMI_baseline_leiden, Purity_baseline_leiden = do_Leiden(G, G, res, 0, label)
                Num_Cluster_Leiden_on_G = len(set(label_))

                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, _, _ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Louv_on_H = len({x for x in set(H_label) if x != -1})
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    # list of unselected_nodes
                    unselected_nodes = [node_idx for node_idx, node_label in enumerate(H_label) if node_label == -1]
                    selected_node_num = G.number_of_nodes() - len(unselected_nodes)
                    '''
                    Random walks from unselected nodes
                    '''
                    H_label, NMI_rand_walk, Purity_rand_walk = do_random_walks(G, unselected_nodes, label, H_label, node2FR, walk_len=10, walk_rep = 100) 
                    selected_node_num2 = len([x for x in H_label if x != -1])
                    selected_node_percent = str(round(selected_node_num*100/(G.number_of_nodes()),2))+'%'
                    #print('selected_before_randwalk:',selected_node_num, 'selected_after_randwalk:',selected_node_num2)
                    '''
                    Excel Export
                    '''
                    data_for_excel[res][FR_tp[FR_type]] = [round(NMI_rand_walk,2), round(Purity_rand_walk,2), selected_node_percent, Num_Cluster_Louv_on_H]
                    
                    '''
                    Plots
                    '''
                    #at index [res][FR_type] store the data for the plot
                    data_for_plot[idx][FR_type] = {'NMI_after_vote':NMI_List[-1], 'Purity_after_vote':Purity_List[-1], 'Louvain_NMI':NMI_, 
                                                    'Louvain_Purity':Purity_, 'Leiden_NMI':NMI_baseline_leiden, 'Leiden_Purity':Purity_baseline_leiden, 
                                                    'Num_Cluster_Louv_on_G':Num_Cluster_Louv_on_G, 'Num_Cluster_Leiden_on_G':Num_Cluster_Leiden_on_G, 
                                                    'Num_Cluster_Louv_on_H':Num_Cluster_Louv_on_H, 'percent_nodes':(selected_node_num*100 / G.number_of_nodes()),
                                                    'extra_nmi':NMI_rand_walk, 'extra_purity':Purity_rand_walk,
                                                    'extra_name':'+RandWalk', 'extra_percent_nodes':(selected_node_num2*100 / G.number_of_nodes())
                                                    }
                    
    plot_to_pdf(resolution_list, data_for_plot, k, True_num_of_clusters, pdf_name, data)
    write_to_excel(data_for_excel, 'results.xlsx', 'results')
        
            

        

### Combined

In [ ]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 
import copy

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq','miRNA']
data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = copy.deepcopy(G)
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    

    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    #data_for_excel['fr_tp'] = FR_tp
    data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 0
    data_for_excel['method_name'] = 'Louv_baseline'
    data_for_excel['num_comm'] = True_num_of_clusters

    data_for_excel2 = copy.deepcopy(data_for_excel)
    data_for_excel3 = copy.deepcopy(data_for_excel)
    data_for_excel3['fr_tp'] = FR_tp
    data_for_excel4 = copy.deepcopy(data_for_excel3)
    data_for_excel5 = copy.deepcopy(data_for_excel3)
    data_for_excel6 = copy.deepcopy(data_for_excel3)
    data_for_excel_ = copy.deepcopy(data_for_excel3)
    data_for_excel7 = copy.deepcopy(data_for_excel3)  

    list_of_excel_data = [data_for_excel, data_for_excel2, data_for_excel3, data_for_excel4, data_for_excel5, data_for_excel_, data_for_excel6, data_for_excel7]
    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            ''' 
            Louvain on G (baseline)
            '''
            label_, NMI_, Purity_ = do_Louvain(G, G, res, 0, label)
            Num_Cluster_Louv_on_G = len(set(label_))
            
            data_for_excel[res]['FL'] = [round(NMI_,2), round(Purity_,2), '1', '100%', Num_Cluster_Louv_on_G]
            
            '''
            Leiden on G (Baseline)
            '''
            label_, NMI_, Purity_ = do_Leiden(G, G, res, 0, label)
            Num_Cluster_Leiden_on_G = len(set(label_))
            
            
            data_for_excel2['method_idx'] = 1
            data_for_excel2['method_name'] = 'Leiden_baseline'
            data_for_excel2[res]['FL'] = [round(NMI_,2), round(Purity_,2), '1', '100%', Num_Cluster_Leiden_on_G]
            


            for Mod_type in [0]:
                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, NMI_, Purity_ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Leid_on_H = len({x for x in set(H_label) if x != -1})

                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())
                    
                    data_for_excel3['fr_tp'] = FR_tp
                    data_for_excel3['method_idx'] = 2
                    data_for_excel3['method_name'] = 'Leiden on 20%'
                    data_for_excel3[res][FR_tp[FR_type]] = [round(NMI_,2), round(Purity_,2), round(preservation_,2),'20%', Num_Cluster_Leid_on_H]
                   
                    '''
                    Add nodes to the subgraph by strong majority vote (1 round)
                    '''
                    H_label_copy = copy.deepcopy(H_label)
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label_copy, G, label, selected_labels_dict, cluster_sizes, one_round=True)

                    selected_labels_dict = defaultdict(int)
                    for node in [idx_ for idx_, x in enumerate(H_label_copy) if x != -1]:
                        selected_labels_dict[label[node]]+=1
                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())

                    selected_node_num = len([x for x in H_label_copy if x != -1])
                    selected_node_percent = str(round(selected_node_num*100/(G.number_of_nodes()),2))+'%'
                    
                    data_for_excel4['method_idx'] = 3
                    data_for_excel4['method_name'] = 'Leiden + Vote(1_Round)'
                    #data_for_excel4[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), selected_node_percent, Num_Cluster_Leid_on_H]
                    data_for_excel4[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), round(preservation_,2), selected_node_percent, Num_Cluster_Leid_on_H]
                  
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num = len([x for x in H_label if x != -1])
                    selected_node_percent = str(round(selected_node_num*100/(G.number_of_nodes()),2))+'%'

                    selected_labels_dict = defaultdict(int)
                    for node in [idx_ for idx_, x in enumerate(H_label) if x != -1]:
                        selected_labels_dict[label[node]]+=1
                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())

                    
                    data_for_excel5['method_idx'] = 4
                    data_for_excel5['method_name'] = 'Leiden + Vote'
                    #data_for_excel5[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), selected_node_percent, Num_Cluster_Leid_on_H]
                    data_for_excel5[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), round(preservation_,2), selected_node_percent, Num_Cluster_Leid_on_H]
                 
                    '''
                    Random walks from unselected nodes
                    '''
                    unselected_nodes = [node_idx for node_idx, node_label in enumerate(H_label) if node_label == -1]
                    H_label_copy = copy.deepcopy(H_label)
                    H_label_copy, NMI_rand_walk, Purity_rand_walk = do_random_walks(G, unselected_nodes, label, H_label_copy, node2FR, walk_len=10, walk_rep = 100) 
                    selected_node_num2 = len([x for x in H_label_copy if x != -1])
                    selected_node_percent2 = str(round(selected_node_num2*100/(G.number_of_nodes()),2))+'%'
                    
                    selected_labels_dict = defaultdict(int)
                    for node in [idx_ for idx_, x in enumerate(H_label_copy) if x != -1]:
                        selected_labels_dict[label[node]]+=1
                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())

                    
                    data_for_excel_['method_idx'] = 5
                    data_for_excel_['method_name'] = 'Leid+Vote+RW(weakMajority)'
                    #data_for_excel_[res][FR_tp[FR_type]] = [round(NMI_rand_walk,2), round(Purity_rand_walk,2), selected_node_percent2, Num_Cluster_Leid_on_H]
                    data_for_excel_[res][FR_tp[FR_type]] = [round(NMI_rand_walk,2), round(Purity_rand_walk,2), round(preservation_,2), selected_node_percent2, Num_Cluster_Leid_on_H]
              
                    '''
                    Leiden again on the vote subgraph
                    '''
                    selected_node_list = [node_num_ for node_num_, node_label_ in enumerate(H_label) if node_label_ != -1]
                    H2 = getInducedSubgraph(G, selected_node_list)
                    H2_ig, ig_to_nx, nx_to_ig = networkx_to_igraph(H2) # Mapping = igraph index to networkx index
                    initial_membership = [H_label[ig_to_nx[i]] for i in list(H2_ig.vs.indices)]
                    partition2_ = la.find_partition(H2_ig, la.RBConfigurationVertexPartition, initial_membership = initial_membership, n_iterations=-1, resolution_parameter=res, seed=0)
                    H_label2 = part_to_full_label(partition2_, G.number_of_nodes(), ig_to_nx)
                    
                    label_compressed2, H_label_compressed2 = get_compressed_labels(label,H_label2, selected_node_list)
                    NMI_louv_vote_louv = NMI(H_label_compressed2, label_compressed2)
                    Purity_louv_vote_louv = met.purity_score(label_compressed2, H_label_compressed2)
                    Num_Cluster_Leid_on_H2 = len({x for x in set(H_label2) if x != -1})
                    
                    selected_labels_dict = defaultdict(int)
                    for node in [idx_ for idx_, x in enumerate(H_label2) if x != -1]:
                        selected_labels_dict[label[node]]+=1
                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())

                    
                    data_for_excel6['method_idx'] = 6
                    data_for_excel6['method_name'] = 'Leid+Vote+Leid'
                    #data_for_excel6[res][FR_tp[FR_type]] = [round(NMI_louv_vote_louv,2), round(Purity_louv_vote_louv,2), selected_node_percent, Num_Cluster_Leid_on_H2]
                    data_for_excel6[res][FR_tp[FR_type]] = [round(NMI_louv_vote_louv,2), round(Purity_louv_vote_louv,2), round(preservation_,2), selected_node_percent, Num_Cluster_Leid_on_H2]
                 
                    '''
                    Leiden + vote + Leiden + vote
                    '''
                    nodes_rest2 = [node_num_ for node_num_, node_label_ in enumerate(H_label2) if node_label_ == -1]
                    NMI_List2, Purity_List2, Balance_List2, Preserv_List2, InEdge_List2 = merge_by_vote(selected_node_list, nodes_rest2, H_label2, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num2 = len([node_num_ for node_num_, node_label_ in enumerate(H_label2) if node_label_ != -1])
                    selected_node_percent = str(round(selected_node_num2*100/(G.number_of_nodes()),2))+'%' 

                    selected_labels_dict = defaultdict(int)
                    for node in [idx_ for idx_, x in enumerate(H_label2) if x != -1]:
                        selected_labels_dict[label[node]]+=1
                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())

                   
                    data_for_excel7['method_idx'] = 7
                    data_for_excel7['method_name'] = '(Leiden + Vote)x2'
                    #data_for_excel7[res][FR_tp[FR_type]] = [round(NMI_List2[-1],2), round(Purity_List2[-1],2), selected_node_percent, Num_Cluster_Leid_on_H2]
                    data_for_excel7[res][FR_tp[FR_type]] = [round(NMI_List2[-1],2), round(Purity_List2[-1],2), round(preservation_,2), selected_node_percent, Num_Cluster_Leid_on_H2]
                   
    
    for excel_data in list_of_excel_data:
        write_to_excel(excel_data, 'results.xlsx', 'results')             
    
        
            

                                
                    

                    
                    



### Combined (With Modularity computation)

In [1]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 
import debug
from collections import defaultdict
import leidenalg as la 
import copy

#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core']
#data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
#data_names = ['Zhengmix8eq','miRNA']
#data_names = ['Zhengmix8eq']
#data_names = ['Zhengmix8eq','mRNA','miRNA','FashionMNIST','MNIST','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Cora','Cora full','Citeseer','Eu core','ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']
data_names = ['ALM', 'AMB', 'Baron_Human', 'Baron_Mouse', 'Muraro', 'Segerstolpe', 'Tcell-medicine', 'TM', 'VISP', 'Xin', 'Zheng']

'''
datasets with nodes that have no outgoing edges : 
['Cora full', 'Citeseer', 'Eu core']
'''

FR_tp = ['FL','FL_ng','FL_ng_pr','PageRank']
FR_tp_short = ['FL','FL_ng','FL_ng_p','PR']
Num_hops = {0:'5',1:'log(n)'}

#top k% nodes for induced subgraph
k=0.2
resolution_list = [0.05, 0.5, 1, 1.5, 5, 50]

for data_idx, data in enumerate(data_names): 

    data_directory = f'./Plots/{data}'
    file_path = f'{data_directory}/{data}.txt'
    os.makedirs(data_directory, exist_ok=True)
    pdf_name = './Plots/'+data+'/Testing'+data+'.pdf'

    '''
    Get the Graph from Edge_List and get the labels
    '''
    G, label= data_to_graph(data)

    True_num_of_clusters = len(set(label))
    

    cluster_sizes = defaultdict(int)
    for node in G.nodes():
        cluster_sizes[label[node]]+=1

    '''
    Add self loops to the nodes with no outgoing edges
    '''
    G_ = copy.deepcopy(G)
    for node in G_.nodes():
        if not list(G_.successors(node)):
            G_.add_edge(node, node, weight=1)

    '''
    dictionary for exporting to excel
    '''
    

    data_for_excel = defaultdict(lambda: defaultdict(int)) #nested dictionary to store data for plots
    data_for_excel['data_idx'] = data_idx
    data_for_excel['data_name'] = data
    #data_for_excel['fr_tp'] = FR_tp
    data_for_excel['fr_tp']=['FL']
    data_for_excel['res_list'] = resolution_list
    data_for_excel['method_idx'] = 0
    data_for_excel['method_name'] = 'Louv_baseline'
    data_for_excel['num_comm'] = True_num_of_clusters

    data_for_excel2 = copy.deepcopy(data_for_excel)
    data_for_excel3 = copy.deepcopy(data_for_excel)
    data_for_excel3['fr_tp'] = FR_tp
    data_for_excel4 = copy.deepcopy(data_for_excel3)
    data_for_excel5 = copy.deepcopy(data_for_excel3)
    data_for_excel6 = copy.deepcopy(data_for_excel3)
    data_for_excel_ = copy.deepcopy(data_for_excel3)
    data_for_excel7 = copy.deepcopy(data_for_excel3)  

    list_of_excel_data = [data_for_excel, data_for_excel2, data_for_excel3, data_for_excel4, data_for_excel5, data_for_excel_, data_for_excel6, data_for_excel7]
    for Num_hop in [1]:
        #List of FR_values
        FR_precalculated = []
        for fr_type in range(4): 
            '''
            Calculate FlowRank for the whole graph
            '''
            if Num_hop==0:
                node2FR = calc_FlowRank(G_, fr_type, 5)
            else:
                node2FR = calc_FlowRank(G_, fr_type, np.log2(G.number_of_nodes()))
            FR_precalculated.append(node2FR)

        for idx, res in enumerate(resolution_list):
            ''' 
            Louvain on G (baseline)
            '''
            label_, NMI_, Purity_ = do_Louvain(G, G, res, 0, label)
            Num_Cluster_Louv_on_G = len(set(label_))
            
            data_for_excel[res]['FL'] = [round(NMI_,2), round(Purity_,2), '1', '100%', Num_Cluster_Louv_on_G]
            data_for_excel[res]['FL'].extend([calc_modularity(G, label_, G, list(G.nodes()), res, label)])

            '''
            Leiden on G (Baseline)
            '''
            label_, NMI_, Purity_ = do_Leiden(G, G, res, 0, label)
            Num_Cluster_Leiden_on_G = len(set(label_))
            
            
            data_for_excel2['method_idx'] = 1
            data_for_excel2['method_name'] = 'Leiden_baseline'
            data_for_excel2[res]['FL'] = [round(NMI_,2), round(Purity_,2), '1', '100%', Num_Cluster_Leiden_on_G]
            data_for_excel2[res]['FL'].extend([calc_modularity(G, label_, G, list(G.nodes()), res, label)])


            for Mod_type in [0]:
                for FR_type in range(4):
                    
                    '''
                    Get induced subgraph of top k% nodes in FR value
                    '''
                    node2FR = FR_precalculated[FR_type]
                    node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                    
                    k_ = int(k*len(node_list))
                    top_nodes = node_list[:k_]
                    nodes_rest = node_list[k_:]
                    H = getInducedSubgraph(G, top_nodes) # H = induced subgraph
                    
                    '''
                    label counts for balancedness calculation
                    '''
                    selected_labels_dict = defaultdict(int)
                    for node in H.nodes():
                        selected_labels_dict[label[node]]+=1
                    
                    '''
                    Leiden on the subgraph H 
                    '''
                    H_label, NMI_, Purity_ = do_Leiden(H, G, res, 0, label)
                    
                    Num_Cluster_Leid_on_H = len({x for x in set(H_label) if x != -1})

                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())
                    
                    data_for_excel3['fr_tp'] = FR_tp
                    data_for_excel3['method_idx'] = 2
                    data_for_excel3['method_name'] = 'Leiden on 20%'
                    data_for_excel3[res][FR_tp[FR_type]] = [round(NMI_,2), round(Purity_,2), round(preservation_,2),'20%', Num_Cluster_Leid_on_H]
                    
                    '''
                    Add nodes to the subgraph by strong majority vote (1 round)
                    '''
                    H_label_copy = copy.deepcopy(H_label)
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label_copy, G, label, selected_labels_dict, cluster_sizes, one_round=True)

                    selected_labels_dict = defaultdict(int)
                    for node in [idx_ for idx_, x in enumerate(H_label_copy) if x != -1]:
                        selected_labels_dict[label[node]]+=1
                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())

                    selected_node_num = len([x for x in H_label_copy if x != -1])
                    selected_node_percent = str(round(selected_node_num*100/(G.number_of_nodes()),2))+'%'
                    
                    data_for_excel4['method_idx'] = 3
                    data_for_excel4['method_name'] = 'Leiden + Vote(1_Round)'
                    #data_for_excel4[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), selected_node_percent, Num_Cluster_Leid_on_H]
                    data_for_excel4[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), round(preservation_,2), selected_node_percent, Num_Cluster_Leid_on_H]
                  
                    '''
                    Add nodes to the subgraph by strong majority vote
                    '''
                    NMI_List, Purity_List, Balance_List, Preserv_List, InEdge_List = merge_by_vote(top_nodes, nodes_rest, H_label, G, label, selected_labels_dict, cluster_sizes)
                    selected_nodes = [idx for idx,x in enumerate(H_label) if x != -1]
                    selected_node_percent = str(round(len(selected_nodes)*100/(G.number_of_nodes()),2))+'%'

                    selected_labels_dict = defaultdict(int)
                    for node in [idx_ for idx_, x in enumerate(H_label) if x != -1]:
                        selected_labels_dict[label[node]]+=1
                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())

                    
                    data_for_excel5['method_idx'] = 4
                    data_for_excel5['method_name'] = 'Leiden + Vote'
                    #data_for_excel5[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), selected_node_percent, Num_Cluster_Leid_on_H]
                    data_for_excel5[res][FR_tp[FR_type]] = [round(NMI_List[-1],2), round(Purity_List[-1],2), round(preservation_,2), selected_node_percent, Num_Cluster_Leid_on_H]
                    
                    H_ = getInducedSubgraph(G, selected_nodes)
                    data_for_excel5[res][FR_tp[FR_type]].extend([calc_modularity(H_, H_label, G, node_list, res, label)])
                    '''
                    Random walks from unselected nodes
                    '''
                    unselected_nodes = [node_idx for node_idx, node_label in enumerate(H_label) if node_label == -1]
                    H_label_copy = copy.deepcopy(H_label)
                    H_label_copy, NMI_rand_walk, Purity_rand_walk = do_random_walks(G, unselected_nodes, label, H_label_copy, node2FR, walk_len=10, walk_rep = 100) 
                    selected_node_num2 = len([x for x in H_label_copy if x != -1])
                    selected_node_percent2 = str(round(selected_node_num2*100/(G.number_of_nodes()),2))+'%'
                    
                    selected_labels_dict = defaultdict(int)
                    for node in [idx_ for idx_, x in enumerate(H_label_copy) if x != -1]:
                        selected_labels_dict[label[node]]+=1
                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())

                    
                    data_for_excel_['method_idx'] = 5
                    data_for_excel_['method_name'] = 'Leid+Vote+RW(weakMajority)'
                    #data_for_excel_[res][FR_tp[FR_type]] = [round(NMI_rand_walk,2), round(Purity_rand_walk,2), selected_node_percent2, Num_Cluster_Leid_on_H]
                    data_for_excel_[res][FR_tp[FR_type]] = [round(NMI_rand_walk,2), round(Purity_rand_walk,2), round(preservation_,2), selected_node_percent2, Num_Cluster_Leid_on_H]
              
                    '''
                    Leiden again on the vote subgraph
                    '''
                    selected_node_list = [node_num_ for node_num_, node_label_ in enumerate(H_label) if node_label_ != -1]
                    H2 = getInducedSubgraph(G, selected_node_list)
                    H2_ig, ig_to_nx, nx_to_ig = networkx_to_igraph(H2) # Mapping = igraph index to networkx index
                    initial_membership = [H_label[ig_to_nx[i]] for i in list(H2_ig.vs.indices)]
                    partition2_ = la.find_partition(H2_ig, la.RBConfigurationVertexPartition, initial_membership = initial_membership, n_iterations=-1, resolution_parameter=res, seed=0)
                    H_label2 = part_to_full_label(partition2_, G.number_of_nodes(), ig_to_nx)
                    
                    label_compressed2, H_label_compressed2 = get_compressed_labels(label,H_label2, selected_node_list)
                    NMI_louv_vote_louv = NMI(H_label_compressed2, label_compressed2)
                    Purity_louv_vote_louv = met.purity_score(label_compressed2, H_label_compressed2)
                    Num_Cluster_Leid_on_H2 = len({x for x in set(H_label2) if x != -1})
                    
                    selected_labels_dict = defaultdict(int)
                    for node in [idx_ for idx_, x in enumerate(H_label2) if x != -1]:
                        selected_labels_dict[label[node]]+=1
                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())

                    
                    data_for_excel6['method_idx'] = 6
                    data_for_excel6['method_name'] = 'Leid+Vote+Leid'
                    #data_for_excel6[res][FR_tp[FR_type]] = [round(NMI_louv_vote_louv,2), round(Purity_louv_vote_louv,2), selected_node_percent, Num_Cluster_Leid_on_H2]
                    data_for_excel6[res][FR_tp[FR_type]] = [round(NMI_louv_vote_louv,2), round(Purity_louv_vote_louv,2), round(preservation_,2), selected_node_percent, Num_Cluster_Leid_on_H2]
                 
                    '''
                    Leiden + vote + Leiden + vote
                    '''
                    nodes_rest2 = [node_num_ for node_num_, node_label_ in enumerate(H_label2) if node_label_ == -1]
                    NMI_List2, Purity_List2, Balance_List2, Preserv_List2, InEdge_List2 = merge_by_vote(selected_node_list, nodes_rest2, H_label2, G, label, selected_labels_dict, cluster_sizes)
                    selected_node_num2 = len([node_num_ for node_num_, node_label_ in enumerate(H_label2) if node_label_ != -1])
                    selected_node_percent = str(round(selected_node_num2*100/(G.number_of_nodes()),2))+'%' 

                    selected_labels_dict = defaultdict(int)
                    for node in [idx_ for idx_, x in enumerate(H_label2) if x != -1]:
                        selected_labels_dict[label[node]]+=1
                    preservation_ = calc_preservation(selected_labels_dict, cluster_sizes, G.number_of_nodes())

                   
                    data_for_excel7['method_idx'] = 7
                    data_for_excel7['method_name'] = '(Leiden + Vote)x2'
                    #data_for_excel7[res][FR_tp[FR_type]] = [round(NMI_List2[-1],2), round(Purity_List2[-1],2), selected_node_percent, Num_Cluster_Leid_on_H2]
                    data_for_excel7[res][FR_tp[FR_type]] = [round(NMI_List2[-1],2), round(Purity_List2[-1],2), round(preservation_,2), selected_node_percent, Num_Cluster_Leid_on_H2]
                   
    
    for excel_data in list_of_excel_data:
        write_to_excel_with_modularity(excel_data, 'results_with_modularity.xlsx', 'results')             
    
        
            

                                
                    

                    
                    



ALM 10068
Log transform done
(10068, 50)
Accuracy of  15 -NN graph is 0.700
151020
AMB 12832
Log transform done
(12832, 50)
Accuracy of  15 -NN graph is 0.763
192480
Baron_Human 8569
Log transform done
(8569, 50)
Accuracy of  15 -NN graph is 0.972
128535
Baron_Mouse 1886
Log transform done
(1886, 50)
Accuracy of  15 -NN graph is 0.954
28290
Muraro 2122
Log transform done
(2122, 50)
Accuracy of  15 -NN graph is 0.950
31830
Segerstolpe 2133
Log transform done
(2133, 50)
Accuracy of  15 -NN graph is 0.933
31995
Tcell-medicine 5759
Log transform done
(5759, 50)
Accuracy of  15 -NN graph is 0.735
86385
TM 54865
Log transform done
(54865, 50)
Accuracy of  15 -NN graph is 0.949
822975
VISP 15413
Log transform done
(15413, 50)
Accuracy of  15 -NN graph is 0.711
231195
Xin 1449
Log transform done
(1449, 50)
Accuracy of  15 -NN graph is 0.974
21735
Zheng 65943
Log transform done
(65943, 50)
Accuracy of  15 -NN graph is 0.561
989145


### Freeze pane

In [2]:
from subgraph import *
freeze_panes('results_with_modularity.xlsx')

### Col Width Adjustment

In [3]:
from subgraph import *
adjust_col_width('results_with_modularity.xlsx')